In [1]:
from zlib import DEF_BUF_SIZE
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from matplotlib.pyplot import figure
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
import mysql.connector
from mysql.connector import Error

import plotly.graph_objects as go
import warnings

from sqlalchemy import false
warnings.filterwarnings("ignore")


def etl(table_name):
   """
       Boman.AI Database Connection from Azure Data base with configuration of user name and password and database name
       Parameters:
           table_name: We have to Pass the Table Name to fetech from DB
   """
   try:
       bomandb = mysql.connector.connect(host='boman-db.mysql.database.azure.com',
                                database='boman_dev',
                                user='bomanadmin@boman-db',
                                password='W$&c614isb#kLlFyJJ9gAE8WGH*!$$$boo')
       # SELECT * FROM boman_dev.sast_results where tool_name='Bandit';
       sast_data = "SELECT * FROM boman_dev.{};".format(table_name) # I ahve to pass either sast_results or dast_results
       data = pd.read_sql(sast_data,bomandb)
       cursor = bomandb.cursor()
       cursor.execute(sast_data)
       records = cursor.fetchall()

   except Error as e :
       print ("Error connecting MySQL", e)
   finally:
       #closing database connection.
       if(bomandb .is_connected()):
           bomandb.close()
           print("MySQL connection is closed Now")
   return data

C:\Users\sumeruinfra\AppData\Local\Temp\ipykernel_24080\3297946238.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\sumeruinfra\AppData\Local\Temp\ipykernel_24080\3297946238.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
sast_tn = etl('sast_results')
dast_tn = etl('dast_results')
sca_tn = etl('sca_results')
sc_tn = etl('secret_scan_results')

MySQL connection is closed Now
MySQL connection is closed Now
MySQL connection is closed Now
MySQL connection is closed Now


In [3]:
sast_tn_vuln_name = dict(sast_tn.vuln_name.value_counts())
dast_tn_vuln_name = dict(dast_tn.vuln_name.value_counts())
sca_tn_vuln_name = dict(sca_tn.vuln_name.value_counts())
sc_tn_vuln_name = dict(sc_tn.vuln_name.value_counts())

In [4]:
di = {}

In [5]:
di.update(sast_tn_vuln_name)
di.update(dast_tn_vuln_name)
di.update(sca_tn_vuln_name)
di.update(sc_tn_vuln_name)

In [6]:
vn = list(di.keys())
vn[0:5]

['hardcoded password string',
 'Remote Code Execution',
 'Mass Assignment',
 'Cross-Site Scripting',
 'SQL Injection']

In [7]:
val = list(di.values())

In [8]:
sast_date,dast_date,sca_date,ss_date =  sast_tn.created_at,dast_tn.created_at,sca_tn.created_at,sc_tn.created_at

In [9]:
Created_date= [sast_date,dast_date,sca_date,ss_date]
Created_date[0][0:6]

0   2022-03-01 08:47:28
1   2022-03-01 08:47:28
2   2022-03-01 08:47:28
3   2022-03-01 08:47:28
4   2022-03-01 08:47:28
5   2022-03-01 08:47:28
Name: created_at, dtype: datetime64[ns]

In [10]:
data = dict({'Vulnerabilities Names':vn,
        "Count":val,
        })

In [11]:
df = pd.DataFrame(data)
df.head()

,Vulnerabilities Names,Count
0,hardcoded password string,240
1,Remote Code Execution,208
2,Mass Assignment,104
3,Cross-Site Scripting,104
4,SQL Injection,104


In [12]:
top10 = df.nlargest(n=10, columns=['Count'])
top10

,Vulnerabilities Names,Count
45,Timestamp Disclosure - Unix,596
46,Information Disclosure - Suspicious Comments,526
47,Content Security Policy (CSP) Header Not Set,490
48,Missing Anti-clickjacking Header,490
49,Absence of Anti-CSRF Tokens,488
50,"Server Leaks Version Information via ""Server"" ...",479
51,X-Content-Type-Options Header Missing,473
52,Modern Web Application,311
0,hardcoded password string,240
1,Remote Code Execution,208


In [13]:
px.pie(top10, names= 'Vulnerabilities Names', values= 'Count',hole=.2)

In [14]:
sast_tn.head(2)

,vuln_id,scan_details_id,scan_token,tool_name,customer_id,vuln_name,vuln_loc,affected_file,vuln_description,vuln_severity,...,tool_version,checklist,more_info,errors,tool_type,ml_status,customer_status,fixed_at,created_at,updated_at
0,439,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,Remote Code Execution,6,app/controllers/password_resets_controller.rb,`Marshal.load` called with parameter value,,...,5.2.1,"BasicAuth,BasicAuthTimingAttack,CSRFTokenForge...",https://brakemanscanner.org/docs/warning_types...,,SAST,1,0,2022-03-27 18:30:45,2022-03-01 08:47:28,2022-03-16 13:36:23
1,440,1,93e567c0-16e1-47b8-be38-bb2669403bb9,Brakeman,1,SQL Injection,29,app/controllers/users_controller.rb,Possible SQL injection,,...,5.2.1,"BasicAuth,BasicAuthTimingAttack,CSRFTokenForge...",https://brakemanscanner.org/docs/warning_types...,,SAST,1,0,2022-03-27 18:30:45,2022-03-01 08:47:28,2022-03-16 13:36:25


In [15]:
df.Count.shape

(146,)

In [45]:
date = pd.DataFrame()
date['Issues_created'] = sast_tn.created_at
date['Issues_fixed'] = sast_tn.fixed_at
date['Solving_time'] = date.Issues_fixed-date.Issues_created 
date['Update_date'] = sast_tn.updated_at
date['Severity'] = sast_tn.vuln_severity
date['Issues_Name'] = sast_tn.vuln_name

In [46]:
date.head()

,Issues_created,Issues_fixed,Solving_time,Update_date,Severity,Issues_Name
0,2022-03-01 08:47:28,2022-03-27 18:30:45,26 days 09:43:17,2022-03-16 13:36:23,,Remote Code Execution
1,2022-03-01 08:47:28,2022-03-27 18:30:45,26 days 09:43:17,2022-03-16 13:36:25,,SQL Injection
2,2022-03-01 08:47:28,2022-03-27 18:30:45,26 days 09:43:17,2022-03-16 13:36:27,,SQL Injection
3,2022-03-01 08:47:28,2022-03-27 18:30:45,26 days 09:43:17,2022-03-16 13:36:29,,Remote Code Execution
4,2022-03-01 08:47:28,2022-03-27 18:30:45,26 days 09:43:17,2022-03-16 13:36:31,,Mass Assignment


In [49]:
date.Severity.unique()

array(['', 'HIGH', 'LOW', 'MEDIUM', 'INFO', '5'], dtype=object)

In [19]:
# Year Wise
by_year = pd.to_datetime(date['Issues_created']).dt.strftime('%Y').value_counts().sort_index()
#by_year.index = pd.PeriodIndex(by_year.index)
by_year = by_year.rename_axis('Yearly').reset_index(name='counts')
by_year

,Yearly,counts
0,2022,1410


In [20]:
px.line(by_year,x='Yearly',y='counts',title='Yearly Issues Creations',color='Yearly',markers=True)

In [21]:
# quarter Wise
by_quarter = pd.to_datetime(date['Issues_created']).dt.to_period('Q').value_counts().sort_index()
by_quarter.index = pd.PeriodIndex(by_quarter.index)
by_quarter = by_quarter.rename_axis('quarter').reset_index(name='counts')
by_quarter

,quarter,counts
0,2022Q1,339
1,2022Q2,1071


In [22]:
fig = go.Figure(data=go.Scatter(x=by_quarter['quarter'].astype(dtype=str), 
                                y=by_quarter['counts'],
                                marker_color='black'))
fig.show()

In [23]:
# Month Wise
by_month = pd.to_datetime(date['Issues_created']).dt.strftime('%m').value_counts().sort_index()
by_month = by_month.rename_axis('month').reset_index(name='counts')
by_month

,month,counts
0,03,339
1,04,1016
2,05,40
3,06,15


In [24]:
#px.line(by_month,x='month',y='counts',title='Monthly Wise Issues Created',markers=True,color='month')

#px.line(by_weak, x='weekly', y="counts",title='Weekly Wise Issues Created',markers=True)

fig = go.Figure(data=go.Scatter(x=by_month['month'].astype(dtype=str), 
                                y=by_month['counts'],
                                marker_color='blue'))
fig.show()

In [25]:
# Weekly wise
by_weak = pd.to_datetime(date['Issues_created']).dt.strftime('%A').value_counts().sort_index()
#by_weak.index = pd.PeriodIndex(by_weak.index)
by_weak = by_weak.rename_axis('weekly').reset_index(name='counts')
by_weak

,weekly,counts
0,Friday,444
1,Monday,275
2,Saturday,69
3,Sunday,138
4,Thursday,79
5,Tuesday,300
6,Wednesday,105


In [26]:
#px.line(by_weak, x='weekly', y="counts",title='Weekly Wise Issues Created',markers=True)

fig = go.Figure(data=go.Scatter(x=by_weak['weekly'].astype(dtype=str), 
                                y=by_weak['counts'],
                                marker_color='black'))
fig.show()

In [27]:
# Day wise
by_day = pd.to_datetime(date['Issues_created']).dt.strftime('%d').value_counts().sort_index()
#by_day.index = pd.PeriodIndex(by_day.index)
by_day = by_day.rename_axis('Days').reset_index(name='counts')
by_day.head()

,Days,counts
0,01,105
1,02,35
2,03,5
3,04,158
4,05,35


In [28]:
px.line(by_day,x='Days',y='counts')

In [29]:
by_date = pd.Series(date['Issues_created']).value_counts().sort_index()
by_date.index = pd.DatetimeIndex(by_date.index)
df_date = by_date.rename_axis('Issues_created').reset_index(name='counts')
df_date

,Issues_created,counts
0,2022-03-01 08:47:28,18
1,2022-03-01 11:36:17,5
2,2022-03-01 12:47:42,1
3,2022-03-01 12:47:43,17
4,2022-03-01 16:42:43,18
...,...,...
150,2022-05-02 16:44:41,4
151,2022-05-02 17:15:49,5
152,2022-05-04 09:58:21,5
153,2022-05-05 14:15:34,5


In [30]:
top_dates = df_date.sort_values(by=['counts'],ascending=False).head(3)
vals = []
for tgl, tot in zip(top_dates["Issues_created"], top_dates["counts"]):
    tgl = tgl.strftime("%d %B")
    val = "%d (%s)"%(tot, tgl)
    vals.append(val)
top_dates['tgl'] = vals
top_dates

,Issues_created,counts,tgl
35,2022-03-29 09:38:02,22,22 (29 March)
39,2022-03-30 04:37:32,22,22 (30 March)
0,2022-03-01 08:47:28,18,18 (01 March)


In [31]:
fig = go.Figure(data=go.Scatter(x=df_date['Issues_created'].astype(dtype=str), 
                                y=df_date['counts'],
                                marker_color='black', text="counts"))
fig.update_layout({"title": 'Top three peak dates',
                   "xaxis": {"title":"Time"},
                   "yaxis": {"title":"Total tweets"},
                   "showlegend": False})
fig.add_traces(go.Scatter(x=top_dates['Issues_created'], y=top_dates['counts'],
                          textposition='top left',
                          textfont=dict(color='#233a77'),
                          mode='markers+text',
                          marker=dict(color='red', size=3),
                          text = top_dates["tgl"]))
fig.show()

In [32]:
px.line(date,x='Issues_created',y='Issues', color = 'Issues')

### Repeated Vulnerabilities across applications
Find the vulnerabilities of each application with each developer (Common across all the applications)

- **ex:** 5 dev is there and 10 apps we have found top 10 Repeated vulnerabilities Names in this app 

In [33]:
app = etl('app')

MySQL connection is closed Now


In [34]:
count = app.app_name.value_counts()

In [35]:
px.line(count)

#### Vulnerabilities Name and There Severity 

In [36]:
def issue_name_mean_min_max(df,vs='vuln_severity'):
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['fixed_at'] = pd.to_datetime(df['fixed_at'],errors='coerce')

    filter_results = df.loc[~df['fixed_at'].isnull(),['vuln_name','fixed_at','created_at',vs]]
    filter_results['MTTR'] = filter_results['fixed_at']-filter_results['created_at']

    final_result = filter_results.groupby(['vuln_name','created_at']).agg({'vuln_name':'count','MTTR':['mean','max','min']})
    final_result.reset_index(inplace=True)
    final_result.columns = ['Vulnerabilities Names','Issue Created','Vulnerabilities count','MTTR','MAXTTR','MINTTR']
    final_result = final_result.sort_values(by='Vulnerabilities count',ascending=False)
    return final_result

In [37]:
sast = issue_name_mean_min_max(sast_tn)
sast.head()

,Vulnerabilities Names,Issue Created,Vulnerabilities count,MTTR,MAXTTR,MINTTR
102,PEAR.Functions.FunctionCallSignature.CloseBrac...,2022-03-27 19:40:33,4,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
117,Remote Code Execution,2022-03-01 18:44:34,4,25 days 23:46:11,25 days 23:46:11,25 days 23:46:11
112,Remote Code Execution,2022-03-01 08:47:28,4,26 days 09:43:17,26 days 09:43:17,26 days 09:43:17
122,Remote Code Execution,2022-03-25 13:49:54,4,2 days 04:40:51,2 days 04:40:51,2 days 04:40:51
115,Remote Code Execution,2022-03-01 16:42:43,4,26 days 01:48:02,26 days 01:48:02,26 days 01:48:02


In [38]:
px.bar(sast,x='Issue Created',y='Vulnerabilities count',color='Vulnerabilities Names')

In [39]:
dast = issue_name_mean_min_max(dast_tn,vs='risk_desc')
dast.head()

,Vulnerabilities Names,Issue Created,Vulnerabilities count,MTTR,MAXTTR,MINTTR
50,Information Disclosure - Suspicious Comments,2022-03-21 04:53:19,12,6 days 13:37:40,6 days 13:37:40,6 days 13:37:40
49,Information Disclosure - Suspicious Comments,2022-03-01 18:47:09,12,25 days 23:43:50,25 days 23:43:50,25 days 23:43:50
51,Information Disclosure - Suspicious Comments,2022-03-23 14:25:46,12,4 days 04:05:13,4 days 04:05:13,4 days 04:05:13
77,Timestamp Disclosure - Unix,2022-03-01 18:47:09,12,25 days 23:43:50,25 days 23:43:50,25 days 23:43:50
78,Timestamp Disclosure - Unix,2022-03-21 04:53:19,12,6 days 13:37:40,6 days 13:37:40,6 days 13:37:40


In [40]:
sca = issue_name_mean_min_max(sca_tn)
sca.head()

,Vulnerabilities Names,Issue Created,Vulnerabilities count,MTTR,MAXTTR,MINTTR
398,CVE-2020-11022,2022-04-05 04:41:38,2,0 days,0 days,0 days
108,CVE-2015-9251,2022-04-04 13:36:02,2,0 days,0 days,0 days
301,CVE-2019-11358,2022-04-04 16:01:29,2,0 days,0 days,0 days
302,CVE-2019-11358,2022-04-04 16:46:34,2,0 days,0 days,0 days
303,CVE-2019-11358,2022-04-04 18:46:07,2,0 days,0 days,0 days


In [41]:
def issue_name_vuln_names_with_mean_min_max_(df,vs='boman_severity'):
    df['created_at'] = pd.to_datetime(df['created_at'])
    df['fixed_at'] = pd.to_datetime(df['fixed_at'],errors='coerce')

    filter_results = df.loc[~df['fixed_at'].isnull(),['vuln_name','fixed_at','created_at','boman_severity']]
    filter_results['MTTR'] = filter_results['fixed_at']-filter_results['created_at']

    final_result = filter_results.groupby(['created_at','vuln_name','boman_severity']).agg({'vuln_name':'count','MTTR':['mean','max','min']})
    final_result.reset_index(inplace=True)
    final_result.columns = ['Issues Created Date','Vulnerabilities Names','Vulnerabilities severity','Count','MTTR','MAXTTR','MINTTR']
    final_result = final_result.sort_values(by='Count',ascending=False)
    return final_result

In [105]:
sast_tool = issue_name_vuln_names_with_mean_min_max_(sast_tn)
sast_tool.head(2)

,Issues Created Date,Vulnerabilities Names,Vulnerabilities severity,Count,MTTR,MAXTTR,MINTTR
0,2022-04-04 13:29:52,hardcoded password string,LOW,1,0 days 00:03:41,0 days 00:03:41,0 days 00:03:41
1,2022-04-04 15:53:55,start process with a shell,HIGH,1,0 days 00:03:56,0 days 00:03:56,0 days 00:03:56


In [ ]:
px.bar(sast_tool,x='Issues Created Date',y='Count',color='Vulnerabilities severity')

### 